In [1]:
import piexif

In [2]:
xf = piexif.load('data/2018-10-03T22-39-25.636580_d6e69720-6603-4191-955b-a310ef6ce9ac/2018-10-03T22-39-25.636580_d6e69720-6603-4191-955b-a310ef6ce9ac.jpg')

In [3]:
xf

{'0th': {270: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  271: b'thl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  272: b'thl T6C\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  274: 1,
  282: (72, 1),
  283: (72, 1),
  296: 2,
  305: b'MediaTek Camera Application\n\x00\x00\x00',
  306: b'2018:10:03 22:39:06',
  531: 2,
  34665: 414},
 'Exif': {33434: (39997, 1000000),
  33437: (28, 10),
  34850: 0,
  34855: 247,
  36864: b'0220',
  36867: b'2018:10:03 22:39:06',
  36868: b'2018:10:03 22:39:06',
  37121: b'\x01\x02\x03\x00',
  37380: (0, 10),
  37383: 2,
  37384: 255,
  37385: 1,
  37386: (350, 100),
  37520: b'5',
  37521: b'5',
  37522: b'5',
  40960: b'0100',
  40961: 1,
  40962: 2448,
  40963: 3264,
  40965: 918,
  41986: 0,
  41987: 0,
  41988: (100, 100),
  41990: 0},
 'GPS'

In [4]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS


class ImageMetaData(object):
    '''
    Extract the exif data from any image. Data includes GPS coordinates, 
    Focal Length, Manufacture, and more.
    '''
    exif_data = None
    image = None

    def __init__(self, img_path):
        self.image = Image.open(img_path)
        #print(self.image._getexif())
        self.get_exif_data()
        super(ImageMetaData, self).__init__()

    def get_exif_data(self):
        """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
        exif_data = {}
        info = self.image._getexif()
        if info:
            for tag, value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t, t)
                        gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        self.exif_data = exif_data
        return exif_data

    def get_if_exist(self, data, key):
        if key in data:
            return data[key]
        return None

    def convert_to_degress(self, value):

        """Helper function to convert the GPS coordinates 
        stored in the EXIF to degress in float format"""
        d0 = value[0][0]
        d1 = value[0][1]
        d = float(d0) / float(d1)

        m0 = value[1][0]
        m1 = value[1][1]
        m = float(m0) / float(m1)

        s0 = value[2][0]
        s1 = value[2][1]
        s = float(s0) / float(s1)

        return d + (m / 60.0) + (s / 3600.0)

    def get_lat_lng(self):
        """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
        lat = None
        lng = None
        exif_data = self.get_exif_data()
        #print(exif_data)
        if "GPSInfo" in exif_data:      
            gps_info = exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = self.convert_to_degress(gps_latitude)
                if gps_latitude_ref != "N":                     
                    lat = 0 - lat
                lng = self.convert_to_degress(gps_longitude)
                if gps_longitude_ref != "E":
                    lng = 0 - lng
        return lat, lng

In [5]:
imd = ImageMetaData('data/2018-10-03T22-39-25.636580_d6e69720-6603-4191-955b-a310ef6ce9ac/2018-10-03T22-39-25.636580_d6e69720-6603-4191-955b-a310ef6ce9ac.jpg')

In [6]:
imd.exif_data

{'ImageDescription': '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'Make': 'thl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'Model': 'thl T6C\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'Orientation': 1,
 'YCbCrPositioning': 2,
 'XResolution': (72, 1),
 'YResolution': (72, 1),
 544: 0,
 545: 0,
 546: 0,
 547: 0,
 548: 0,
 549: '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'ResolutionUnit': 2,
 'ExifOffset': 414,
 'Software': 'MediaTek Camera Application\n\x00\x00\x00',
 'DateTime': '2018:10:03 22:39:06',
 'ExifVersion': b'0220',
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'FlashPixVersion': b'0100',
 'DateTimeOriginal': '2018:10:03 22:39:06',
 'DateTimeDigitized': '2018:10:03 22:39:06',
 'Expo

In [7]:
for tag in imd.exif_data:
    print(tag)

ImageDescription
Make
Model
Orientation
YCbCrPositioning
XResolution
YResolution
544
545
546
547
548
549
ResolutionUnit
ExifOffset
Software
DateTime
ExifVersion
ComponentsConfiguration
FlashPixVersion
DateTimeOriginal
DateTimeDigitized
ExposureBiasValue
ColorSpace
MeteringMode
LightSource
Flash
FocalLength
ExifImageWidth
ExifImageHeight
ExifInteroperabilityOffset
SceneCaptureType
SubsecTime
SubsecTimeOriginal
SubsecTimeDigitized
ExposureTime
FNumber
ExposureProgram
ISOSpeedRatings
ExposureMode
WhiteBalance
DigitalZoomRatio


In [8]:
imd.exif_data['Flash']

1

In [9]:
dd = {"r": 5, "g": 100, "b":200}

In [10]:
dd['red']

KeyError: 'red'

In [19]:
import reload

ModuleNotFoundError: No module named 'reload'

In [27]:
import caas.lib
import importlib
importlib.reload(caas.lib)

<module 'caas.lib' from '/Users/michaelbach/Documents/GitHub/app1/_testing/post/caas/lib.py'>

In [28]:
wp = caas.lib.WorkingPathsAndDirectoriesForIcoming()

AttributeError: 'PurePosixPath' object has no attribute 's'

In [29]:
print(wp)

uuid: a18277ba-ce07-4a6c-aa8a-2651bc9ef2ba
timestamp: 2018-10-06T15-19-58.876034
path_and_filename_to_incoming_image: data/2018-10-06T15-19-58.876034_<module 'uuid' from '/Users/michaelbach/anaconda3/lib/python3.6/uuid.py'>/2018-10-06T15-19-58.876034_<module 'uuid' from '/Users/michaelbach/anaconda3/lib/python3.6/uuid.py'>.jpg
path_to_incoming_image: data/2018-10-06T15-19-58.876034_<module 'uuid' from '/Users/michaelbach/anaconda3/lib/python3.6/uuid.py'>
path_to_working_dir_analysis: data/2018-10-06T15-19-58.876034_<module 'uuid' from '/Users/michaelbach/anaconda3/lib/python3.6/uuid.py'>/2018-10-06T15-19-58.876034_analysis
path_to_working_dir_training: data/2018-10-06T15-19-58.876034_<module 'uuid' from '/Users/michaelbach/anaconda3/lib/python3.6/uuid.py'>/2018-10-06T15-19-58.876034_analysis

